# new method to reflector.

**Input:** Set of pairs $ P(p,m) $, optimization index $ n $  
**Output:** Optimized set of pairs $ P' $

1. $ P' \leftarrow P $
2. if $ n = \text{Length}(P') $ then
3. &nbsp;&nbsp;&nbsp;&nbsp;return $ P' $
4. end if
5. $ P'_1 \leftarrow \text{OPO}(P', n + 1) $
6. if $ \text{Split}(P'[n]) \neq \emptyset $ then
7. &nbsp;&nbsp;&nbsp;&nbsp;(p1, m1), (p2, m2) $ \leftarrow \text{Split}(P'[n]) $
8. &nbsp;&nbsp;&nbsp;&nbsp;$ P' \leftarrow P' \cup \{(p1, m1), (p2, m2)\} $
9.  &nbsp;&nbsp;&nbsp;&nbsp;Delete $ P'[n] $
10. &nbsp;&nbsp;&nbsp;&nbsp;$ P'_2 \leftarrow \text{OPO}(P', n) $
11. end if
12. return $ \text{Better}(P'_1, P'_2) $

first to design the operation of permutation, here use the 32-bit register, 16-cell, 2-block, for the permutation. we want let the result is the sequence operation of the shift & or & and, and have the operation register. the start register is r0, the operation temp register is r1, the result register is r14

In [8]:
# aes permutation
permutation = [0, 1, 2, 3, 7, 4, 5, 6, 10, 11, 8, 9, 13, 14, 15, 12]
# transform to sequence operation
sequence = []


def PPO(sequence, temp, origin, dest, mask, shift):
    sequence.append(("and", temp, origin, hex(mask)))
    sequence.append(("ror", temp, temp, shift))
    sequence.append(("or", dest, dest, temp))


for i in range(len(permutation)):
    # first need to mask to r1,
    # then self-shift r1, there have the ror operation, so all the shift direction is right,
    # if the shift is left, need to use the length 32 sub the shift value.
    # then or operation to r14
    mask = 2 << ((15 - i) * 2)
    distance = i - permutation.index(i)
    shift = 2 * (distance) if distance >= 0 else 2 * (distance + 16)
    PPO(sequence, "r1", "r0", "r14", mask, shift)
print(sequence)
print(len(sequence))
# here is not optimal sequence, so need to optimize the sequence

[('and', 'r1', 'r0', '0x80000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x20000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x8000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x2000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x800000'), ('ror', 'r1', 'r1', 30), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x200000'), ('ror', 'r1', 'r1', 30), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x80000'), ('ror', 'r1', 'r1', 30), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x20000'), ('ror', 'r1', 'r1', 6), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x8000'), ('ror', 'r1', 'r1', 28), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x2000'), ('ror', 'r1', 'r1', 28), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x800'), ('ror', 'r1', 'r1', 4), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x200'), ('ror', 'r1', 'r1', 4), ('or', '

one pop operation is including the and, ror, or operation, that format `[('and', 'r1', 'r0', '0x80000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1')]`

if only the mask value is not same, then can merge the operation, merge is xor the mask value.

In [ ]:
def merge_sequence(sequence):
    # Group operations by 3 (one POP = and+ror+or)
    pops = [sequence[i : i + 3] for i in range(0, len(sequence), 3)]
    merged = []
    processed = set()

    for i, pop1 in enumerate(pops):
        if i in processed:
            continue

        mask1 = int(pop1[0][3], 16)  # Convert hex mask to int
        shift1 = int(pop1[1][3])
        current_group = [pop1]

        # Find pops with different masks that can be merged
        for j, pop2 in enumerate(pops[i + 1 :], i + 1):
            if j in processed:
                continue

            mask2 = int(pop2[0][3], 16)
            shift2 = int(pop2[1][3])
            # Check if masks are different
            if mask1 != mask2 and shift1 == shift2:
                # Merge masks using XOR
                mask1 ^= mask2
                current_group.append(pop2)
                processed.add(j)

        if len(current_group) > 1:
            # Create merged operation
            merged.extend(
                [
                    ("and", "r1", "r0", hex(mask1)),
                    ("ror", "r1", "r1", current_group[0][1][3]),
                    ("or", "r14", "r14", "r1"),
                ]
            )
        else:
            # Keep original operation
            merged.extend(pop1)

    return merged


merged = merge_sequence(sequence)
print(merged)
print(len(merged))

[('and', 'r1', 'r0', '0xaa000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0xa80000'), ('ror', 'r1', 'r1', 30), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x20000'), ('ror', 'r1', 'r1', 6), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0xa000'), ('ror', 'r1', 'r1', 28), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0xa00'), ('ror', 'r1', 'r1', 4), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x80'), ('ror', 'r1', 'r1', 26), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x2a'), ('ror', 'r1', 'r1', 2), ('or', 'r14', 'r14', 'r1')]
21
